<a href="https://colab.research.google.com/github/HuntressM/HuntressM/blob/main/song_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install spotipy

In [ ]:
pip install scikit-learn

In [ ]:
import spotipy
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from spotipy.oauth2 import SpotifyClientCredentials
from sklearn.metrics import pairwise_distances_argmin_min

In [ ]:
# While we don't have access to spotify, we are using a previously saved data set

df = pd.read_csv('songs_data.csv', index_col='Unnamed: 0')
df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,artist,song name
0,0.677,0.696,0,-6.181,1,0.1190,0.3210,0.000000,0.4200,0.4640,90.030,audio_features,4iJyoBOLtHqaGxP12qzhQI,spotify:track:4iJyoBOLtHqaGxP12qzhQI,https://api.spotify.com/v1/tracks/4iJyoBOLtHqa...,https://api.spotify.com/v1/audio-analysis/4iJy...,198082,4,Justin Bieber,Peaches (feat. Daniel Caesar & Giveon)
1,0.623,0.668,6,-7.282,0,0.0531,0.4750,0.000000,0.0862,0.8230,161.948,audio_features,6n9DThpbZpZF9A5K3kysa4,spotify:track:6n9DThpbZpZF9A5K3kysa4,https://api.spotify.com/v1/tracks/6n9DThpbZpZF...,https://api.spotify.com/v1/audio-analysis/6n9D...,183067,4,Justin Bieber,Mistletoe
2,0.623,0.668,6,-7.282,0,0.0531,0.4750,0.000000,0.0862,0.8230,161.948,audio_features,7kz4V8L9MiGdDPcGi4F2j3,spotify:track:7kz4V8L9MiGdDPcGi4F2j3,https://api.spotify.com/v1/tracks/7kz4V8L9MiGd...,https://api.spotify.com/v1/audio-analysis/7kz4...,183067,4,Justin Bieber,Mistletoe
3,0.638,0.515,10,-8.575,1,0.0288,0.5210,0.000000,0.3170,0.4190,100.010,audio_features,6lMV2oCyNSf5zy3GrWjqXy,spotify:track:6lMV2oCyNSf5zy3GrWjqXy,https://api.spotify.com/v1/tracks/6lMV2oCyNSf5...,https://api.spotify.com/v1/audio-analysis/6lMV...,206800,4,Justin Bieber,Christmas Love
4,0.533,0.426,9,-9.525,0,0.0297,0.4600,0.000000,0.1000,0.1520,103.267,audio_features,4ujZCIL3YzGzeIby5SZ8it,spotify:track:4ujZCIL3YzGzeIby5SZ8it,https://api.spotify.com/v1/tracks/4ujZCIL3YzGz...,https://api.spotify.com/v1/audio-analysis/4ujZ...,216427,3,Justin Bieber,The Christmas Song (Chestnuts Roasting On An O...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,0.324,0.110,9,-19.780,0,0.0351,0.0847,0.777000,0.6210,0.0393,123.534,audio_features,3QEWEv1tvHmVgsT5THpGDn,spotify:track:3QEWEv1tvHmVgsT5THpGDn,https://api.spotify.com/v1/tracks/3QEWEv1tvHmV...,https://api.spotify.com/v1/audio-analysis/3QEW...,255934,3,pink floyd,Don't Leave Me Now
146,0.280,0.716,9,-12.095,0,0.0983,0.0652,0.520000,0.5120,0.0618,135.378,audio_features,55e99bv2wlgTYtNEYHWtN1,spotify:track:55e99bv2wlgTYtNEYHWtN1,https://api.spotify.com/v1/tracks/55e99bv2wlgT...,https://api.spotify.com/v1/audio-analysis/55e9...,619079,4,pink floyd,Sheep
147,0.691,0.324,0,-15.366,1,0.0446,0.1710,0.022300,0.0674,0.5430,121.664,audio_features,3kbUFmLV5SHVQonfOpNgkd,spotify:track:3kbUFmLV5SHVQonfOpNgkd,https://api.spotify.com/v1/tracks/3kbUFmLV5SHV...,https://api.spotify.com/v1/audio-analysis/3kbU...,223067,4,pink floyd,San Tropez
148,0.287,0.499,0,-11.094,1,0.0437,0.4640,0.000652,0.1020,0.1420,176.437,audio_features,1bQ4zbP8Fo0qAePTSOfYji,spotify:track:1bQ4zbP8Fo0qAePTSOfYji,https://api.spotify.com/v1/tracks/1bQ4zbP8Fo0q...,https://api.spotify.com/v1/audio-analysis/1bQ4...,379241,4,pink floyd,Coming Back to Life


In [ ]:
# When you have access to spotify you can get your own songs:

#Initialize SpotiPy with user credentias
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id="af3a4e21d9974f798b0ddef081728f2b",
                                                           client_secret="99a65d20eff04d64bcf24b11824dffc4"))

results = sp.search(q='artist:Justin Bieber', limit=50)


In [ ]:
# extract the track ids
track_ids = [track['id'] for track in results['tracks']['items']]

# extract the audio features
audio_features = sp.audio_features(track_ids)

# store audio features in a dataframe
df = pd.DataFrame(audio_features)

In [ ]:
# create a function to automate work
def get_audio_features(artist):
    # get tracks from artist
    results = sp.search(q=f'artist:{artist}', limit=50)
    # extract the track ids
    track_ids = [track['id'] for track in results['tracks']['items']]
    song_names = [track['name'] for track in results['tracks']['items']]
    # extract the audio features
    audio_features = sp.audio_features(track_ids)
    # store audio features in a dataframe
    df = pd.DataFrame(audio_features)
    df['artist'] = artist
    df['song name'] = song_names
    return df

In [ ]:
# create loop and select artists you want to get songs from

artists = ['Justin Bieber', 'Nirvana', 'pink floyd']

df = pd.DataFrame()

for artist in artists:
    df_artist = get_audio_features(artist)
    df = pd.concat([df, df_artist])
    
df = df.reset_index(drop=True)

In [ ]:
x = df[['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']]
y = df['artist']

x_prep = StandardScaler().fit_transform(x)

kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(x_prep)

clusters = kmeans.predict(x_prep)
df['cluster'] = clusters

In [ ]:
# analyzing our results, we can start to see some winners
df.groupby(['cluster', 'artist'], as_index=False).count().sort_values(['cluster', 'index'], ascending=False)[['cluster', 'artist', 'index']]

,cluster,artist,index
7,2,pink floyd,38
6,2,Nirvana,3
3,1,Justin Bieber,47
4,1,Nirvana,15
5,1,pink floyd,8
1,0,Nirvana,32
2,0,pink floyd,4
0,0,Justin Bieber,3


Check out how to visualize your clusters [here](https://towardsdatascience.com/how-to-build-an-amazing-music-recommendation-system-4cce2719a572).

In [ ]:
# Toxic - Britney Spears
# Yellow Submarine - The Beatles
# Stairway to Heaven - Led Zeppelin
# Wonderwall - Oasis
# It's my life - Bon Jovi
# Californication - Red Hot Chilli Peppers

song_name = input('Choose a song: ')
results = sp.search(q=f'track:{song_name}', limit=1)
track_id = results['tracks']['items'][0]['id']
audio_features = sp.audio_features(track_id)
df_ = pd.DataFrame(audio_features)
x = df_[['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']]

Choose a song: Helena - My Chemical Romance


In [ ]:
closest, _ = pairwise_distances_argmin_min(x, df[x.columns])
closest

array([41])

In [ ]:
df.loc[closest]['song name'], df.loc[closest]['artist']

(41    What Do You Mean?
 Name: song name, dtype: object,
 41    Justin Bieber
 Name: artist, dtype: object)

#### Next Steps:

In [ ]:
# put everything inside a function
def recommend_song():
    song_name = input('Choose a song: ')
    results = sp.search(q=f'track:{song_name}', limit=1)
    track_id = results['tracks']['items'][0]['id']
    audio_features = sp.audio_features(track_id)
    df_ = pd.DataFrame(audio_features)
    x = df_[['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']]
    closest, _ = pairwise_distances_argmin_min(x, df[x.columns])
    return ' - '.join([df.loc[closest]['song name'].values[0], df.loc[closest]['artist'].values[0]])

In [ ]:
recommend_song()

Choose a song: It's my life - Bon Jovi


'Smells Like Teen Spirit - Nirvana'

In [ ]:
# TODO: Get more songs, Evaluate your results

![](https://c.tenor.com/GYswjo8j1QsAAAAC/dungeons-dragons-80s.gif)